In [0]:
pip install pyspark

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
spark.conf.set(
    "fs.azure.account.key.datacohortworkspacelabs.dfs.core.windows.net",
    "NUwhjFcHG95EU1Rnu7+Woq3JQP28bXy5kDQhA9yFV68XBz1umr7uqeQgMhrwxHfTwNWxAx/n1K6j+AStGTUMkQ==")

In [0]:
from pyspark.sql.functions import col

silver_path = "abfss://silver-kay@datacohortworkspacelabs.dfs.core.windows.net/"
gold_path = "abfss://gold-kay@datacohortworkspacelabs.dfs.core.windows.net/"

silver_data = spark.read.format("delta").load(silver_path)

PULocID = silver_data.select("PULocationID").distinct()
DOLocID = silver_data.select("DOLocationID").distinct()

LocationID = PULocID.union(DOLocID).distinct().withColumnRenamed("PULocationID", "LocationID")

trips_fact = silver_data.select("tripID", "VendorID", "PULocationID", "DOLocationID", "coverageID", "trip_distance", "fareID", "total_amount", "fare_amount", "tip_amount").limit(1000000)
vendors_dim = silver_data.select("VendorID").distinct().limit(1000000)
locations_dim = LocationID.limit(1000000)
coverage_dim = silver_data.select("coverageID", "tpep_pickup_datetime", "tpep_dropoff_datetime").distinct().limit(1000000)
fare_dim = silver_data.select("fareID", "payment_type").distinct().limit(1000000)

trips_fact.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save(gold_path + "/trips_fact")
vendors_dim.write.format("delta").mode("overwrite").save(gold_path + "/vendors_dim")
locations_dim.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save(gold_path + "/locations_dim")
coverage_dim.write.format("delta").mode("overwrite").save(gold_path + "/coverage_dim")
fare_dim.write.format("delta").mode("overwrite").save(gold_path + "/fare_dim")


In [0]:
gold_path = "abfss://gold-kay@datacohortworkspacelabs.dfs.core.windows.net/"

table_names = [
    "coverage_dim",
    "vendors_dim", 
    "fare_dim", 
    "locations_dim", 
    "trips_fact"
]

for table_name in table_names:
    data_path = f"{gold_path}{table_name}/"
    table_data = spark.read.format("delta").option("header", "true").load(data_path)
   

    table_data.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(f"gold_pbix_kay.{table_name}")

In [0]:
num_rows = trips_fact.count()

print(f"The DataFrame has {num_rows} rows.")


The DataFrame has 1000000 rows.
